# Security Constrained OPF (SCOPF)

Finds least-cost dispatch satisfying base-case and N-1 constraints.

Import the case and instantiate the `PowerWorld`.

```python
from esapp import PowerWorld
from esapp.components import *

pw = PowerWorld(case_path)
```

In [ ]:
# This cell is hidden in the documentation.
from esapp import PowerWorld
from esapp.components import *
import matplotlib.pyplot as plt
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_dual_bar

## Setup SCOPF Optimization

Initialize the solver and prepare contingency constraints for the security-constrained problem:

The Primal LP solver is PowerWorld's optimization engine. Auto-insert N-1 contingencies to make the optimization security-constrained:

In [ ]:
# Capture pre-OPF dispatch
pre_opf = pw[Gen, ['BusNum', 'GenMW', 'GenStatus']]
pre_opf_online = pre_opf[pre_opf['GenStatus'] == 'Closed'].copy()

pw.esa.InitializePrimalLP()
pw.auto_insert_contingencies()

## Solve SCOPF

In [ ]:
pw.esa.SolveFullSCOPF()

production_cost = pw[Area, "GenProdCost"]
print("Production Cost by Area:")
print(production_cost.to_string(index=False))

In [ ]:
post_opf = pw[Gen, ['BusNum', 'GenMW', 'GenStatus']]
post_opf_online = post_opf[post_opf['GenStatus'] == 'Closed'].copy()

fig, axes = plt.subplots(1, 2, figsize=(6.5, 2.8))
plot_dual_bar(pre_opf_online['GenMW'].values, post_opf_online['GenMW'].values,
              label_a='Pre-OPF', label_b='Post-OPF',
              xlabel='Generator Index', ylabel='MW Output',
              title='Dispatch Comparison', ax=axes[0])
# Redispatch delta
delta = post_opf_online['GenMW'].values - pre_opf_online['GenMW'].values
colors = ['#55A868' if d >= 0 else '#C44E52' for d in delta]
axes[1].bar(range(len(delta)), delta, color=colors)
from examples.map import format_plot
format_plot(axes[1], title='Redispatch Delta',
            xlabel='Generator Index', ylabel='\u0394 MW',
            plotarea='white', titlesize=11, labelsize=9, ticksize=8)
plt.tight_layout()
plt.show()